In [1]:
from sklearn.pipeline import Pipeline, FeatureUnion
import sklearn.decomposition as decomposition
from sklearn.decomposition import SparsePCA
import sklearn.preprocessing as preprocessing
from sklearn.feature_selection import SelectKBest
import pandas as pd
import sklearn.preprocessing as pp
import numpy as np
import sklearn.naive_bayes as naive_bayes
import sklearn.cross_validation as cross_validation

# Loading the Data

In [24]:
X = pd.read_csv('../assignment1_2016S1/training_data.csv',header=None).sort_values(by=0).reset_index().ix[:,2:]
y = pd.read_csv('../assignment1_2016S1/training_labels.csv',header=None).sort_values(by=0).reset_index().ix[:,2:]

In [27]:

X_scaled = X.applymap(lambda x: 1 if x > 0 else 0)

# Feature Engineering / Pre-Processing

In [41]:
svd = decomposition.TruncatedSVD(n_components=700,random_state=42)

In [39]:
pca = decomposition.PCA(n_components=0.90)

In [46]:
count = []
for name,series in X_scaled.iteritems():
    if(series.sum() < 500):
        count += [name]
len(count)

13048

In [47]:
temp = pd.DataFrame(X[count])
temp[13627] = temp.sum(axis=1)
temp.shape

(20104, 13049)

# Naive Bayes (Gaussian and Multinomial)

In [49]:
X_train,X_test,y_train,y_test = cross_validation.train_test_split(temp, y, test_size=0.33, random_state=42)

nb = naive_bayes.GaussianNB()
mnb = naive_bayes.MultinomialNB()

raw_pipe = Pipeline([("naive_bayes",nb)])
pca_pipe = Pipeline([("pca",pca),("naive_bayes", nb)])
svd_pipe = Pipeline([("svd",svd),("naive_bayes",nb)])
raw_pipe_mnb = Pipeline([("naive_bayes",mnb)])
pipes = [raw_pipe,pca_pipe,svd_pipe,raw_pipe_mnb]
for pipeline in pipes:
    print(pipeline.fit(X_train,y_train.as_matrix().ravel()).score(X_test,y_test))

0.429238884702
0.188997739261
0.0795779954785
0.503391107762


# Multinomial Naive Bayes

# Decision Tree

# SVM - Linear Kernel

# Random Forest